# 6.2. 图像卷积
## 6.2.1. 互相关运算

In [10]:
import torch
def corr2d(X, K):
    '''
    @desc: 计算二维互相关运算
    @param: X 输入特征 
            K 卷积核
    '''
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [11]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 6.2.2. 卷积层
卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置之后产生输出。
高度和宽度分别为h和w的卷积核可以被称为h*w卷积或h*w卷积核。 我们也将带有h*w卷积核的卷积层称为h*w卷积层

In [12]:
class Conv2D(torch.nn.Module):
    def __init__(self, kernel_size) -> None:
        super().__init__()
        self.weight = torch.nn.Parameter(torch.rand(kernel_size))
        self.bias = torch.nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias


## 6.2.3. 图像中目标的边缘检测
如下是卷积层的一个简单应用：通过找到像素变化的位置，来检测图像中不同颜色的边缘。

In [13]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [14]:
K = torch.Tensor([[1.0, -1.0]])
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [15]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 6.2.4. 学习卷积核
计算梯度来更新卷积核

In [16]:
conv2d = torch.nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2
for i in range(10):
    Y_hat = conv2d(X)
    l = torch.pow(Y_hat - Y, 2).sum()
    conv2d.zero_grad()
    l.backward()

    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    print(f'epoch {i+1}, loss {l:.3f}')

epoch 1, loss 4.573
epoch 2, loss 1.948
epoch 3, loss 0.846
epoch 4, loss 0.377
epoch 5, loss 0.174
epoch 6, loss 0.084
epoch 7, loss 0.042
epoch 8, loss 0.023
epoch 9, loss 0.013
epoch 10, loss 0.007


In [17]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9852, -1.0010]])